In [ ]:
# Copyright 2025 Sony Corporation

## get Max F Value's meta division

In [ ]:
import subprocess
import os
import glob
import shutil
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [ ]:
workdir = os.getcwd()

In [ ]:
fsom_experiment_name = "Flowsom"

In [ ]:
samplenames = ["FP34c","Levine_32dim","Levine_13dim","Samusik_01","Samusik_all",]


In [ ]:
#range of search best F 
#please fix it for fp34c
xrange_dict = {"FP34c": range(14,20,1),
               "Levine_32dim": range(14,37,1), 
               "Levine_13dim": range(24,50,1),
               "Samusik_01": range(24,39,1),
               "Samusik_all":range(24,39,1)
              }

In [ ]:
for sample_name in samplenames:
    print(sample_name)
    fsom_exp_path = os.path.join(workdir,fsom_experiment_name,sample_name) 
    os.chdir(workdir)
    resdf = pd.DataFrame()
    meta_list = xrange_dict.get(sample_name) #samplename -> get range
    for meta in meta_list:
        directory = os.path.join(fsom_exp_path,"%d"%meta,"10")    
        files = [file for file in os.listdir(directory) if (file.startswith("vseed_f1hang_") and file.endswith("resh"))]
        df = pd.DataFrame()
        for file in files:
            file_path = os.path.join(directory, file)
            # Get the numeric part from the file name and use it as an index
            index = int(file.split('_')[2].split('.')[0])
            temp_df = pd.read_csv(file_path,header=None)
            # set index and concatenate
            temp_df['index'] = index
            temp_df.set_index('index', inplace=True)
            df = pd.concat([df, temp_df])
        data_hang = df.rename(columns={0:"%d"%meta})
        resdf = pd.concat([resdf,data_hang],axis=1)
    d = pd.melt(resdf) #flowsom result
    max_index = resdf.mean().sort_values().idxmax()
    max_idx = int(max_index) - 10
    max_index2 = resdf.median().sort_values().idxmax()
    print(max_index)
    m = resdf.mean()
    filename = "%s_meta_F.csv"%sample_name
    m.to_csv(filename)

In [ ]:
BL_experiment_name = "BL_FlowSOM"

In [ ]:
for sample_name in samplenames:
    print(sample_name)
    exp_path = os.path.join(workdir,BL_experiment_name,sample_name) 
    os.chdir(workdir)
    resdf = pd.DataFrame()
    meta_list = xrange_dict.get(sample_name) #samplename -> get range
    data = []
    for meta in meta_list:
        directory = os.path.join(exp_path)    
        file = os.path.join(directory,"f1_meta_%02d_it_10_hang.hres"%meta)
        with open(file,"r") as f:
            value = float(f.readline().strip())
        data.append((meta, value))
    df = pd.DataFrame(data,columns=["meta","Value"])
    filename = "BL_%s_meta_f.csv"%sample_name
    df.to_csv(filename)